In [ ]:
# default_exp core

# tai64converter

> Functions to encode / decode tai64 values to datetime.

Licenced from [Itamar Turner-Trauring](https://github.com/itamarst).

Copyright 2020 Itamar Turner-Trauring

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

   http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
#export
from __future__ import unicode_literals
import struct
from binascii import b2a_hex, a2b_hex
_STRUCTURE = b">QI"
_OFFSET = (2 ** 62) + 10  # last 10 are leap seconds
def _tai_encode(timestamp):
    """
    Copyright 2020 Itamar Turner-Trauring

    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.
    """
    """
    Convert seconds since epoch to TAI64N string.
    @param timestamp: Seconds since UTC Unix epoch as C{float}.
    @return: TAI64N-encoded time, as C{unicode}.
    """
    seconds = int(timestamp)
    nanoseconds = int((timestamp - seconds) * 1000000000)
    seconds = seconds + _OFFSET
    encoded = b2a_hex(struct.pack(_STRUCTURE, seconds, nanoseconds))
    return "@" + encoded.decode("ascii")
def _tai_decode(tai64n):
    """
    Copyright 2020 Itamar Turner-Trauring

    Licensed under the Apache License, Version 2.0 (the "License");
    you may not use this file except in compliance with the License.
    You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing, software
    distributed under the License is distributed on an "AS IS" BASIS,
    WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
    See the License for the specific language governing permissions and
    limitations under the License.
    """
    """
    Convert TAI64N string to seconds since epoch.
    Note that dates before 2013 may not decode accurately due to leap second
    issues. If you need correct decoding for earlier dates you can try the
    tai64n package available from PyPI (U{https://pypi.python.org/pypi/tai64n}).
    @param tai64n: TAI64N-encoded time, as C{unicode}.
    @return: Seconds since UTC Unix epoch as C{float}.
    """
    seconds, nanoseconds = struct.unpack(_STRUCTURE, a2b_hex(tai64n[1:]))
    seconds -= _OFFSET
    return seconds + (nanoseconds / 1000000000.0)

In [ ]:
#export
from datetime import datetime
class Tai64Converter:
    def __init__(self, timestamp=None, datetime=None, tai=None):
        self.t = None
        if timestamp is not None:
            self.t = self.encode(timestamp)
        elif datetime is not None:
            self.t = self.encode(datetime.timestamp())
        elif tai is not None:
            self.t = tai
    def encode(self, value=None):
        if value is not None:
            return _tai_encode(value)
        else:
            assert self.t is not None
            return self.t
    def decode(self, value=None):
        if value is not None:
            return datetime.fromtimestamp(_tai_decode(value))
        else:
            assert self.t is not None
            return datetime.fromtimestamp(_tai_decode(self.t))

In [ ]:
t = '@400000005ebb9bde074d7c28'
d = datetime.strptime('2020-05-13 10:03:48.122518', '%Y-%m-%d %H:%M:%S.%f')
print(d.timestamp())
print(str(d))

1589353428.122518
2020-05-13 10:03:48.122518


In [ ]:
t64 = Tai64Converter()
test_close(t64.decode(t).timestamp(), d.timestamp(), eps=1e-6)

In [ ]:
t64 = Tai64Converter()
test_eq(t64.encode(d.timestamp())[:-3], t[:-3])

In [ ]:
t64 = Tai64Converter(timestamp=d.timestamp())
test_close(t64.decode().timestamp(), d.timestamp(), eps=1e-6)
test_eq(t64.encode()[:-3], t[:-3])

In [ ]:
t64 = Tai64Converter(tai=t)
test_close(t64.decode().timestamp(), d.timestamp(), eps=1e-6)
test_eq(t64.encode()[:-3], t[:-3])

In [ ]:
t64 = Tai64Converter(datetime=d)
test_close(t64.decode().timestamp(), d.timestamp(), eps=1e-6)
test_eq(t64.encode()[:-3], t[:-3])

In [ ]:
t64 = Tai64Converter()
with ExceptionExpected():
    t64.decode()
with ExceptionExpected():
    t64.encode()

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
